In [2]:
#
import os
import sys
import warnings
from pathlib import Path
import dataclasses
from typing import Dict, Union

#
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

# Stable Baseline3
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

# AI4Finance
from meta.config import TIME_ZONE_PARIS

#
import seaborn as sns
from copy import deepcopy
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyfolio
from pyfolio import timeseries
import tqdm
import yfinance as yf
import pyfolio as pf

#
from rl.plot.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from common.utils import now_time
from configuration.settings import ProjectDir, ExperimentDir
from rl.data.CompanyInfo import CompanyInfo
from rl.envs.StockTradingEnv import StockTradingEnv
from rl.experiments._1_same_bigger_data_fundamental.train import (
    Program,
    CustomDRLAgent,
    get_dataset,
    get_env_kwargs,
    dataset_name,
    base_cols,
    data_cols,
    ratios_cols,
    algorithm_name,
)

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
def ignore_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)


ignore_warnings()

In [4]:
def main():
    program = Program(
        prj_dir=ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing")),
        exp_dir=ExperimentDir(Path(os.getcwd())),
        DEBUG=False,
    )
    program.dataset = get_dataset(
        pd.read_csv(program.exp_dir.out.datasets.joinpath(f"{dataset_name}.csv"), index_col=0), purpose="test"
    )
    program.exp_dir.check_and_create_dirs()
    return program


program = main()

In [5]:
print(base_cols)
print(data_cols)
print(ratios_cols)

['date', 'tic']
['open', 'high', 'low', 'close', 'volume']
['operatingProfitMargin', 'netProfitMargin', 'returnOnAssets', 'returnOnEquity', 'currentRatio', 'quickRatio', 'cashRatio', 'inventoryTurnover', 'receivablesTurnover', 'payablesTurnover', 'debtRatio', 'debtEquityRatio', 'priceEarningsRatio', 'priceBookValueRatio', 'dividendYield']


In [6]:
@dataclasses.dataclass
class LearnedAlgorithm:
    algorithm: str
    filename: Path
    learned_algorithm: Union[A2C, PPO, DDPG, A2C, TD3]


def get_algorithm(filename: Path):
    if "a2c" in filename.as_posix():
        return LearnedAlgorithm(algorithm="a2c", filename=filename, learned_algorithm=A2C.load(filename))


learned_algorithms = [get_algorithm(filepath) for filepath in program.exp_dir.out.algorithms.glob("*")]

In [7]:
# Env
env_kwargs = get_env_kwargs(program.dataset)
env_gym = StockTradingEnv(df=program.dataset, **env_kwargs)

Stock Dimension: 29, State Space: 494


In [8]:
# Test Learned Algorithms
df_account_value, df_actions = CustomDRLAgent.DRL_prediction(
    model=learned_algorithms[len(learned_algorithms) - 1].learned_algorithm, environment=env_gym
)

hit end!


In [9]:
df_account_value

,date,account_value
0,2019-04-12,1.000000e+06
1,2019-04-15,9.994271e+05
2,2019-04-16,1.000304e+06
3,2019-04-17,9.962704e+05
4,2019-04-18,9.988517e+05
...,...,...
923,2022-12-09,1.241070e+06
924,2022-12-12,1.260773e+06
925,2022-12-13,1.278515e+06
926,2022-12-14,1.262068e+06


In [10]:
# Get results
perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv(program.exp_dir.out.results.joinpath(f"perf_stats_all_{algorithm_name}_{now_time()}.csv"))

Annual return          0.057717
Cumulative returns     0.229539
Annual volatility      0.295385
Sharpe ratio           0.339129
Calmar ratio           0.130704
Stability              0.362444
Max drawdown          -0.441589
Omega ratio            1.068750
Sortino ratio          0.469728
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.037811
Daily value at risk   -0.036818
dtype: float64


In [11]:
start = program.dataset["date"].min()
end = program.dataset["date"].max()
print(start, end)

2019-04-12 2022-12-15


In [36]:
# Baseline
baseline_tic = yf.Ticker("^DJI")

In [37]:
baseline_df = baseline_tic.history(interval="1d", start=start, end=end)
baseline_df
# baseline_df = get_baseline("^DJI", start, end, "1d") # TODO: fixme

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-04-12 00:00:00-04:00,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
2019-04-15 00:00:00-04:00,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2019-04-16 00:00:00-04:00,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
2019-04-17 00:00:00-04:00,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
2019-04-18 00:00:00-04:00,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
...,...,...,...,...,...,...,...
2022-12-08 00:00:00-05:00,33695.968750,33899.171875,33642.460938,33781.480469,271830000,0,0
2022-12-09 00:00:00-05:00,33746.710938,33844.628906,33462.648438,33476.460938,287630000,0,0
2022-12-12 00:00:00-05:00,33519.500000,34005.039062,33519.500000,34005.039062,313440000,0,0


In [38]:
date_format = "%Y-%m-%d"
baseline_df.index = baseline_df.index.strftime(date_format)
baseline_df.insert(0, "date", baseline_df.index)

In [39]:
baseline_df

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2019-04-12,2019-04-12,26357.789062,26436.679688,26309.720703,26412.300781,369910000,0,0
2019-04-15,2019-04-15,26407.759766,26424.849609,26316.419922,26384.769531,250620000,0,0
2019-04-16,2019-04-16,26482.189453,26530.710938,26397.189453,26452.660156,302340000,0,0
2019-04-17,2019-04-17,26468.529297,26501.019531,26391.859375,26449.539062,367780000,0,0
2019-04-18,2019-04-18,26463.369141,26602.419922,26444.529297,26559.539062,332850000,0,0
...,...,...,...,...,...,...,...,...
2022-12-08,2022-12-08,33695.968750,33899.171875,33642.460938,33781.480469,271830000,0,0
2022-12-09,2022-12-09,33746.710938,33844.628906,33462.648438,33476.460938,287630000,0,0
2022-12-12,2022-12-12,33519.500000,34005.039062,33519.500000,34005.039062,313440000,0,0


In [40]:
print(baseline_df.index.min())
print(baseline_df.index.max())

2019-04-12
2022-12-14


In [41]:
stats = backtest_stats(baseline_df, value_col_name="Close")

Annual return          0.070772
Cumulative returns     0.286005
Annual volatility      0.232728
Sharpe ratio           0.411239
Calmar ratio           0.190831
Stability              0.561379
Max drawdown          -0.370862
Omega ratio            1.087791
Sortino ratio          0.567932
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.953450
Daily value at risk   -0.028941
dtype: float64


In [42]:
backtest_plot(
    df_account_value,
    baseline_ticker="^DJI",
    baseline_start=program.dataset.index.min(),
    baseline_end=program.dataset.index.max(),
)

AttributeError: module 'meta' has no attribute 'data_processors'